<a href="https://colab.research.google.com/github/delaykimm/COSE461-NLP/blob/main/XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 라이브러리 설치 및 임포트

In [ ]:
!pip install accelerate>=0.21.0
!pip install Keras-Preprocessing
!pip install datasets
!pip install transformers==4.17
!pip install wandb

import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup, EarlyStoppingCallback
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers.trainer_callback import TrainerCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstall

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# 파일 경로 설정
train_path = '/content/drive/My Drive/crawling_everytime.csv'
test_path = '/content/drive/My Drive/everytime_Korean.csv'

# 데이터 로드
train_dataset = pd.read_csv(train_path)
test_dataset = pd.read_csv(test_path)

train_dataset = train_dataset[["text", "label"]]
test_dataset = test_dataset.rename(columns={"tweet": "text"})
#print(test_dataset)
test_dataset = test_dataset[["text", "label"]]

# 훈련/검증 데이터 분할
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_dataset["text"].tolist(),
    train_dataset["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=train_dataset["label"].tolist()
)

#train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

print(train_dataset.shape)
print(test_dataset.shape)

Mounted at /content/drive
(14715, 2)
(1000, 2)


### 데이터 로드 및 전처리

In [ ]:
# 1. 모델과 토크나이저 로드
model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name)

# 3. 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length = 128)

# 데이터셋 생성 및 전처리
train_data = {'text': train_texts, 'label': train_labels}
val_data = {'text': val_texts, 'label': val_labels}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])

train_dataset.set_format("torch")
val_dataset.set_format("torch")

# Weights and Biases 초기화
wandb.init(project="depression-detection")

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Map:   0%|          | 0/11772 [00:00<?, ? examples/s]

Map:   0%|          | 0/2943 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
class CustomCallback(TrainerCallback):
    def on_log(self, args, state, control, **kwargs):
        # 각 배치마다 training loss 출력
        if 'loss' in state.log_history[-1]:
            print(f"Step {state.global_step}: training loss = {state.log_history[-1]['loss']}")

        # 각 배치마다 validation loss 출력
        if 'eval_loss' in state.log_history[-1]:
            print(f"Step {state.global_step}: validation loss = {state.log_history[-1]['eval_loss']}")

        # 에포크마다 wandb로 기록
        if state.is_world_process_zero and 'epoch' in state.log_history[-1]:
            wandb.log(state.log_history[-1])

### 훈련 인자 및 trainer 설정

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉토리
    num_train_epochs=3,              # 훈련 에포크 수
    per_device_train_batch_size=64,   # 배치 크기
    per_device_eval_batch_size=64,    # 검증 배치 크기
    warmup_steps=300,                # 학습률 스케줄링을 위한 워밍업 스텝
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로그 디렉토리
    logging_steps=10,
    evaluation_strategy="steps",     # 검증 전략을 "steps"로 설정
    eval_steps=len(train_dataset)//64, # 각 배치마다 검증 수행
    save_steps=len(train_dataset)//64, # 몇 스텝마다 모델 저장할지
    report_to="wandb",               # Weights and Biases에 기록
    load_best_model_at_end=True,     # 가장 좋은 모델을 마지막에 로드
)

total_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

from torch.optim.lr_scheduler import OneCycleLR

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 스케줄러 설정
# scheduler = OneCycleLR(
#     optimizer,
#     max_lr=2e-5,
#     total_steps = total_steps
# )

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #optimizers=(optimizer, scheduler),  # 옵티마이저 및 스케줄러 설정
    callbacks=[CustomCallback(), EarlyStoppingCallback(early_stopping_patience=3)]  # 사용자 정의 콜백 추가
)

# 모델 훈련 및 평가
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11772
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
183,0.557600,0.644774
366,0.566500,0.545932
549,0.527600,0.524213


Step 10: training loss = 0.7168
Step 20: training loss = 0.7015
Step 30: training loss = 0.6726
Step 40: training loss = 0.6691
Step 50: training loss = 0.644
Step 60: training loss = 0.6161
Step 70: training loss = 0.619
Step 80: training loss = 0.5412
Step 90: training loss = 0.5558
Step 100: training loss = 0.5423
Step 110: training loss = 0.5376
Step 120: training loss = 0.5604
Step 130: training loss = 0.5657
Step 140: training loss = 0.5469
Step 150: training loss = 0.5448
Step 160: training loss = 0.5343
Step 170: training loss = 0.5535
Step 180: training loss = 0.5576


***** Running Evaluation *****
  Num examples = 2943
  Batch size = 64


Step 183: validation loss = 0.6447738409042358


...

Step 549: validation loss = 0.524212658405304


Saving model checkpoint to ./results/checkpoint-549
Configuration saved in ./results/checkpoint-549/config.json
Model weights saved in ./results/checkpoint-549/pytorch_model.bin


Step 550: training loss = 0.4871




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-549 (score: 0.524212658405304).


TrainOutput(global_step=552, training_loss=0.5493639003539431, metrics={'train_runtime': 526.0381, 'train_samples_per_second': 67.136, 'train_steps_per_second': 1.049, 'total_flos': 2515207988164608.0, 'train_loss': 0.5493639003539431, 'epoch': 3.0})

### 특정 텍스트에 대한 임베딩 추출

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Hook를 이용해 마지막 은닉층의 출력을 추출할 변수 정의
embeddings_list = []

def hook(module, input, output):
    embeddings_list.append(output)

# 모델의 마지막 은닉층에 hook 등록
hook_handle = model.transformer.word_embedding.register_forward_hook(hook)

# 데이터 로더 설정
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64)

# 임베딩 추출 함수
def extract_embeddings(dataloader, model):
    model.eval()
    embeddings = []
    labels = []
    texts = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to(model.device) for key, val in batch.items() if key != "label"}
            texts.extend(batch['input_ids'].cpu().numpy())
            labels.extend(batch["label"].cpu().numpy())
            model(**inputs)
            last_hidden_state = embeddings_list[-1]  # 마지막 은닉층 출력
            cls_embeddings = last_hidden_state[:, 0, :].cpu().numpy()  # [CLS] 토큰의 임베딩 추출
            embeddings.extend(cls_embeddings)
            embeddings_list.clear()  # 다음 배치를 위해 리스트 초기화
    return np.array(embeddings), np.array(labels), np.array(texts)

# 훈련 및 검증 데이터로부터 임베딩 추출
train_embeddings, train_labels, train_texts = extract_embeddings(train_dataloader, model)
val_embeddings, val_labels, val_texts = extract_embeddings(val_dataloader, model)

# 임베딩을 TensorFlow 형식으로 저장
def save_embeddings(embeddings, labels, texts, embeddings_path, metadata_path):
    np.savetxt(embeddings_path, embeddings, delimiter='\t')
    with open(metadata_path, 'w') as f:
        f.write("Text\tLabel\n")
        for text, label in zip(texts, labels):
            f.write(f"{text}\t{label}\n")

save_embeddings(train_embeddings, train_labels, train_texts, 'train_embeddings.tsv', 'train_metadata.tsv')
save_embeddings(val_embeddings, val_labels, val_texts, 'val_embeddings.tsv', 'val_metadata.tsv')

# Hook 해제
hook_handle.remove()

In [ ]:
from google.colab import files

# 파일 다운로드
files.download('train_embeddings.tsv')
files.download('train_metadata.tsv')
files.download('val_embeddings.tsv')
files.download('val_metadata.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>